In [1]:
from secret import rpc_user, rpc_password

In [2]:
## Check the bitcoind running
import os

datadir = os.path.abspath(os.path.expanduser('~/.bitcoin'))
pid_path = os.path.join(datadir, 'bitcoind.pid')
if os.path.exists(pid_path):
    with open(pid_path, 'r') as f:
        print(f'Bitcoind PID: {f.read()}')
else:
    raise SystemExit('Bitcoind is not running!')

Bitcoind PID: 30675



In [3]:
from address_convertor import pubkey_to_address, get_pubkey

In [4]:
## Some function for convenient
import datetime

tz_seoul = datetime.timezone(datetime.timedelta(hours=9))
tz_utc = datetime.timezone(datetime.timedelta())

def get_time(timestamp):
    return datetime.datetime.fromtimestamp(timestamp, tz=tz_seoul)

In [5]:
import datetime
from bitcoinrpc.authproxy import AuthServiceProxy, JSONRPCException
rpc_ip = '127.0.0.1'
rpc_port = '8332'
timeout = 300

def get_rpc():
    return AuthServiceProxy(f'http://{rpc_user}:{rpc_password}@{rpc_ip}:{rpc_port}', timeout=timeout)

rpc_connection = get_rpc()
best_block_hash = rpc_connection.getbestblockhash()
best_block = rpc_connection.getblock(best_block_hash)
print(f'Best Block Heights: {best_block["height"]}, Time: {get_time(best_block["time"]).isoformat()}')

Best Block Heights: 641278, Time: 2020-07-29T13:50:56+09:00


In [ ]:
import pickle

category_clusters = dict()
with open('./경로', 'rb') as fr:
    category_clusters = pickle.load(fr)

## 특징 연산 코드

In [ ]:
def get_addr_btc_usd_spent(addr_id):
    total_spent_value = 0.0; total_spent_usd = 0.0; convert_susd = 0.0
    cur.execute('''SELECT tx, btc FROM TxOut WHERE addr = ?''', (addr_id,))
    for tx, btc in cur:
        # btc 계산
        total_spent_value += btc
        
        # usd 계산
        cur.execute('''SELECT unixtime FROM BlkTime JOIN BlkTx ON BlkTime.blk = BlkTx.blk
                                WHERE BlkTx.tx = ?''', (tx,))
        get_time = cur.fetchone()[0]
        itime = datetime.datetime.fromtimestamp(get_time).strftime('%Y-%m-%d') # convert to datetime
        try:
            convert_susd = float(list(preprocess_csv[preprocess_csv['date'] == itime]['USD'])[0].replace(",","")) # search usd
        except IndexError:
            convert_susd = float(list(preprocess_csv[preprocess_csv['date'] == '2010-07-18']['USD'])[0].replace(",","")) # search usd
        total_spent_usd += btc * convert_susd
        
    return total_spent_value, total_spent_usd

def get_addr_btc_usd_received(addr_id):
    total_received_value = 0.0 # btc
    total_received_usd = 0.0; convert_susd = 0.0 # usd
    sum_utxo_btc = 0.0 # balance
    
    cur.execute('''SELECT tx, btc FROM TxIn WHERE addr = ?''', (addr_id,))
    for tx, btc in cur:
        # btc 계산
        total_received_value += btc
        
        # usd 계산
        tmp_cur.execute('''SELECT unixtime FROM BlkTime JOIN BlkTx ON BlkTime.blk = BlkTx.blk
                                WHERE BlkTx.tx = ?''', (tx,))
        get_time = cur.fetchone()[0]
        itime = datetime.datetime.fromtimestamp(get_time).strftime('%Y-%m-%d') # convert to datetime
        try:
            convert_susd = float(list(preprocess_csv[preprocess_csv['date'] == itime]['USD'])[0].replace(",","")) # search usd
        except IndexError:
            convert_susd = float(list(preprocess_csv[preprocess_csv['date'] == '2010-07-18']['USD'])[0].replace(",","")) # search usd

        total_received_usd += btc * convert_susd
        
        # balance 계산
        tmp_cur.execute('''SELECT addr, btc FROM TxOut WHERE tx = ?''', (tx,))
        for addr, byc in tmp_cur:
            try:
                tmp_cur2.execute('''SELECT COUNT(addr) FROM TxIn WHERE addr = ?''', (addr,))
                if tmp_cur2.fetchone()[0] == 0:
                    sum_utxo_btc += btc
            except KeyError:
                continue
    
    return total_received_value, total_received_usd, sum_utxo_btc

def get_age(addr_id):
    txhash_list = list()
    
    # 처음 생성은 tx_out 가장 첫번째에서...
    cur.execute('''SELECT BlkTime.unixtime FROM BlkTime JOIN BlkTx ON BlkTime.blk = BlkTx.blk
                            JOIN TxOut ON BlkTx.tx = TxOut.tx WHERE TxOut.addr = ?''', (addr_id,))
    txhash_list.append(cur.fetchone()[0])
    
    # 마지막 생성은 tx_in, tx_out 내림차순으로 정리해서 확인
    cur.execute('''SELECT BlkTime.unixtime FROM BlkTime JOIN BlkTx ON BlkTime.blk = BlkTx.blk 
                    JOIN TxIn ON BlkTx.tx = TxIn.tx WHERE TxIn.addr = ? ORDER BY TxIn.tx desc''', (addr_id,))
    txin_txhash = cur.fetchone()[0]
    cur.execute('''SELECT BlkTime.unixtime FROM BlkTime JOIN BlkTx ON BlkTime.blk = BlkTx.blk 
                    JOIN TxOut ON BlkTx.tx = TxOut.tx WHERE TxOut.addr = ? ORDER BY TxOut.tx desc''', (addr_id,))
    txhash_list.append([txin_txhash, cur.fetchone()[0]])
    
    first_check_time = 0; last_check_time = 0
    for i, time in enumerate(txhash_list):
        if i == 0: # 처음 생성된 시간을 구하는.
            first_check_time = time
        else: # 마지막으로 사용된 시간을 구하는.
            for th_time in tl:                    
                if last_check_time < th_time:
                    last_check_time = th_time
                    
    return last_check_time - first_check_time

def get_fee(addr_id):
    activate_txin = set(); activate_txout = set(); fee_list = list()
    cur.execute('''SELECT tx FROM TxOut WHERE addr = ?''', (addr_id,))
    for row in cur:
        activate_txout.add(row)
    cur.execute('''SELECT tx FROM TxIn WHERE addr = ?''', (addr_id,))
    for row in cur:
        activate_txin.add(row)
    
    txout_value = 0; txin_value = 0
    for atxo in activate_txout:
        tmp_cur.execute('''SELECT btc FROM TxOut WHERE tx = ?''', (atxo[0],))
        for row in tmp_cur:
            txout_value += row[0]
    for atxi in activate_txin:
        tmp_cur.execute('''SELECT btc FROM TxIn WHERE tx = ?''', (atxi[0],))
        for row in tmp_cur:
            txin_value += row[0]
            
    return txin_value - txout_value

def get_number_spent(addr_id):
    cur.execute('''SELECT count(*) FROM TxOut WHERE addr = ?''', (addr_id,))
    return cur.fetchone()[0]

def get_number_received(addr_id):
    cur.execute('''SELECT count(*) FROM TxIn WHERE addr = ?''', (addr_id,))
    return cur.fetchone()[0]